# Memory Registry Analysis
Another area to investigate in memory is the registry data resident which can provide useful informaiton on host configuration, user activity, and malware persistence mechanism. Where is memory Stored

* C:\Windows\System32\config
* Windows 7

The common registry file are:

* SYSTEM
* SOFTWARE
* SECURITY
* NTUSER
* USERCLASS
* SAM

These files are refered to as "hives". These file are not human readable and need to be parsed with special tools. These files are best acquired forensically or data extracted through memory. The registry keeps track of informaiton in Keys and values that contain data. Keys in the registry are analhous to file paths, values to files and data as file content. The SYSTEM and SOFTWARE registry file contain the bulk of the system configuration settings.

The Registry only has one timestamp which is assocated with the last write time of the Key, so if a Key has multiple values and only one changes the last write is only associated with the key not the value or the data. 

You can see where registry is stored in memory by execution the following volatility plugin:

In [1]:
python ../volatility3/vol.py -h

The below script needs to be able to find the current output cell; this is an easy method to get it.

Volatility 3 Framework 1.2.1
usage: volatility [-h] [-c CONFIG] [--parallelism [{processes,threads,off}]]
                  [-e EXTEND] [-p PLUGIN_DIRS] [-s SYMBOL_DIRS] [-v] [-l LOG]
                  [-o OUTPUT_DIR] [-q] [-r RENDERER] [-f FILE]
                  [--write-config] [--clear-cache] [--cache-path CACHE_PATH]
                  [--offline] [--single-location SINGLE_LOCATION]
                  [--stackers [STACKERS [STACKERS ...]]]
                  [--single-swap-locations [SINGLE_SWAP_LOCATIONS [SINGLE_SWAP_LOCATIONS ...]]]
                  plugin ...

An open-source memory forensics framework

optional arguments:
  -h, --help            Show this help message and exit, for specific plugin
                        options use 'volatility <pluginname> --help'
  -c CONFIG, --config CONFIG
                        Load the configuration from a json file
  --parallelism [{processes,threads,off}]
                        Enables parallelism (defaults to off if no argument
       

    windows.envars.Envars
                        Display process environment variables
    windows.filescan.FileScan
                        Scans for file objects present in a particular windows
                        memory image.
    windows.getservicesids.GetServiceSIDs
                        Lists process token sids.
    windows.getsids.GetSIDs
                        Print the SIDs owning each process
    windows.handles.Handles
                        Lists process open handles.
    windows.info.Info   Show OS & kernel details of the memory sample being
                        analyzed.
    windows.malfind.Malfind
                        Lists process memory ranges that potentially contain
                        injected code.
    windows.memmap.Memmap
                        Prints the memory map
    windows.modscan.ModScan
                        Scans for modules present in a particular windows
                        memory image.
    windows.modules.Modules
            

In [2]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.hivelist.HiveList

Volatility 3 Framework 1.2.1

Offset	FileFullPath	File output

0xe1b01718	\Device\HarddiskVolume1\Documents and Settings\demo\Local Settings\Application Data\Microsoft\Windows\UsrClass.dat	Disabled
0xe1a47b60	\Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT	Disabled
0xe190e008	\Device\HarddiskVolume1\Documents and Settings\LocalService\Local Settings\Application Data\Microsoft\Windows\UsrClass.dat	Disabled
0xe1905b60	\Device\HarddiskVolume1\Documents and Settings\LocalService\NTUSER.DAT	Disabled
0xe1830b60	\Device\HarddiskVolume1\Documents and Settings\NetworkService\Local Settings\Application Data\Microsoft\Windows\UsrClass.dat	Disabled
0xe1836b60	\Device\HarddiskVolume1\Documents and Settings\NetworkService\NTUSER.DAT	Disabled
0xe14b56b8	\Device\HarddiskVolume1\WINDOWS\system32\config\software	Disabled
0xe1739710	\Device\HarddiskVolume1\WINDOWS\system32\config\default	Disabled
0xe173eb60	\Device\HarddiskVolume1\WINDOWS\system32\config\SAM	Disabled
0xe14b5b60	\Device\Har


One of the first items to collect when performing analysis is to gather basic host configutation information to ensure you are examining the right system and document what system has what findings. Most of the configuration information needing to be documented comes from the registry such as the following:

* Hostname
* Domain
* IP address
* Network assignment protocol

To get this inforamtion from the registry using volatilty you will use the `printkey` command and the specified registry key where the configuration data resides. The registry stores configuration information in multiple locations and can have multiple copies of this configuration as backup if a system crashes or gets corrupted in some manner. The registry documents what configuration information is current by the key `HKLM\CurrentControlSet`. To get this setting execute the following command:

## Current Control Set

In [6]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "CurrentControlSet"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\CurrentControlSet	-		-
-	0xe1a47b60	Key	?\CurrentControlSet	-		-
-	0xe190e008	Key	?\CurrentControlSet	-		-
-	0xe1905b60	Key	?\CurrentControlSet	-		-
-	0xe1830b60	Key	?\CurrentControlSet	-		-
-	0xe1836b60	Key	?\CurrentControlSet	-		-
-	0xe14b56b8	Key	?\CurrentControlSet	-		-
-	0xe1739710	Key	?\CurrentControlSet	-		-
-	0xe173eb60	Key	?\CurrentControlSet	-		-
-	0xe14b5b60	Key	?\CurrentControlSet	-		-
-	0xe13a9758	Key	?\CurrentControlSet	-		-
2009-04-16 16:09:51.000000 	0xe1035b60	REG_LINK	\Device\HarddiskVolume1\WINDOWS\system32\config\system\CurrentControlSet	SymbolicLinkValue	"\Registry\Machine\System\ControlSet001"	True
-	0xe102e008	Key	?\CurrentControlSet	-		-


You will use this value as input for the rest of the specific `printkey` commands for the active control set. 

## Hostname

In [7]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Control\ComputerName\ComputerName"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe1a47b60	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe190e008	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe1905b60	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe1830b60	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe1836b60	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe14b56b8	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe1739710	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe173eb60	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe14b5b60	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
-	0xe13a9758	Key	?\ControlSet001\Control\ComputerName\ComputerName	-		-
2008-08-11 22:47:29.000000 	0xe1035b60	REG_SZ	\Device\HarddiskVolume1\WINDOWS\system32\config\system\ControlSet001\Contro

## Domain and DHCP Configurations

In [8]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Services\Tcpip\Parameters"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe1a47b60	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe190e008	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe1905b60	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe1830b60	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe1836b60	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe14b56b8	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe1739710	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe173eb60	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe14b5b60	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
-	0xe13a9758	Key	?\ControlSet001\Services\Tcpip\Parameters	-		-
2009-02-17 04:13:18.000000 	0xe1035b60	Key	\Device\HarddiskVolume1\WINDOWS\system32\config\system\ControlSet001\Services\Tcpip\Parameters	Adapters		False
2009-02-17 04:13:18.000000 	0xe1035b60	Key	\Device\Hard

## Network Interfaces

In [9]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Services\Tcpip\Parameters\Interfaces"#

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe1a47b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe190e008	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe1905b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe1830b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe1836b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe14b56b8	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe1739710	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe173eb60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe14b5b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
-	0xe13a9758	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces	-		-
2009-02-16 22:17:22.000000 	0xe1035b60	Key	\Device\HarddiskVolume1\WINDOWS\system32\conf

## Parse information in each Interface

In [10]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe1a47b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe190e008	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe1905b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe1830b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe1836b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe14b56b8	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-	0xe1739710	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{5B00ABC6-D4D3-46B2-9C5D-5618273C35F6}	-		-
-

In [11]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe1a47b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe190e008	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe1905b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe1830b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe1836b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe14b56b8	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-	0xe1739710	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{8E96184A-2A18-4D1C-AA97-4F126156F2EF}	-		-
-

In [12]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe1a47b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe190e008	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe1905b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe1830b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe1836b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe14b56b8	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-	0xe1739710	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{942691E0-1A9A-4F39-B9B4-7D077FC9B26B}	-		-
-

In [13]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe1a47b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe190e008	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe1905b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe1830b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe1836b60	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe14b56b8	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-	0xe1739710	Key	?\ControlSet001\Services\Tcpip\Parameters\Interfaces\{C2513D1F-B099-4423-9C77-A76C525BF24C}	-		-
-

## Timezone

In [14]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "ControlSet001\Control\TimeZoneInformation"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe1a47b60	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe190e008	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe1905b60	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe1830b60	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe1836b60	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe14b56b8	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe1739710	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe173eb60	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe14b5b60	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
-	0xe13a9758	Key	?\ControlSet001\Control\TimeZoneInformation	-		-
2009-03-17 17:07:55.000000 	0xe1035b60	REG_DWORD	\Device\HarddiskVolume1\WINDOWS\system32\config\system\ControlSet001\Control\TimeZoneInformation	Bias	300	False
2009-03-17 17:07:55.000000

### Basic Host Information
|Hostname     |IP     |Domain     |Timezone|
|-------------|-------|-----------|--------|
|victim1|192.168.157.10|N/A|EST|

## Persistence

The registry houses the largest set of persistence mechansims on Windows. There are tens if not hundreds of registry keys which can be abused my malware to maintain persistence. Common persistence mechanisms in the registry are the following:

* HKLM\Software\Microsoft\Windows\CurrentVersion\Run
* HKLM\Software\Microsoft\Windows\CurrentVersion\RunOnce
* HKLM\Microsoft\Windows\CurrentVersion\RunServices
* HKCU\Software\Microsoft\Windows\CurrentVersion\Run
* HKCU\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\Run

These keys can be enumerated from memory by calling each key to review the values. An example of listing these keys content is executed in the following plugin:

In [15]:
python ../volatility3/vol.py -q -f E:\APT.img  windows.registry.printkey.PrintKey --key "Microsoft\Windows\CurrentVersion\Run"

Volatility 3 Framework 1.2.1

Last Write Time	Hive Offset	Type	Key	Name	Data	Volatile

-	0xe1b01718	Key	?\Microsoft\Windows\CurrentVersion\Run	-		-
-	0xe1a47b60	Key	?\Microsoft\Windows\CurrentVersion\Run	-		-
-	0xe190e008	Key	?\Microsoft\Windows\CurrentVersion\Run	-		-
-	0xe1905b60	Key	?\Microsoft\Windows\CurrentVersion\Run	-		-
-	0xe1830b60	Key	?\Microsoft\Windows\CurrentVersion\Run	-		-
-	0xe1836b60	Key	?\Microsoft\Windows\CurrentVersion\Run	-		-
2009-01-27 04:59:59.000000 	0xe14b56b8	REG_SZ	\Device\HarddiskVolume1\WINDOWS\system32\config\software\Microsoft\Windows\CurrentVersion\Run	porkbun	"
43 00 3a 00 5c 00 57 00	C.:.\.W.
49 00 4e 00 44 00 4f 00	I.N.D.O.
57 00 53 00 5c 00 73 00	W.S.\.s.
79 00 73 00 74 00 65 00	y.s.t.e.
6d 00 33 00 32 00 5c 00	m.3.2.\.
65 00 78 00 70 00 6c 00	e.x.p.l.
6f 00 64 00 65 00 72 00	o.d.e.r.
2e 00 65 00 78 00 65 00	..e.x.e.
00 00 00 00 00 00 00 00	........
00 00 00 00 00 00 00 00	........
00 00 00 00 00 00 00 00	........
00 00 00 00 00 00 00 00	........
0

Since there are so many keys to review it can become laborious to review every single key but luckily there are custom developed volatility plugins to help with this.  

### Autoruns
Autoruns is a custom volatility plugin that enumerates most of the Windows registry persistence mechanisms. This plugin will query the possible keys and execute the print plugin for you. This plugin can be found here:

* https://github.com/tomchop/volatility-autoruns

To activate the plugin download the python script and put it into your volatilty source folder in the `plugin` directory. You can use this method to either write your own plugins or download others custom plugins. A link to commonly used and communitiy developed volatility plugins not in the main source can be found here:

* https://github.com/volatilityfoundation/community

To run the `autoruns` plugin execution the following command:

In [13]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 autoruns

Volatility Foundation Volatility Framework 2.6
ERROR   : volatility.debug    : You must specify something to do (try -h)


### Services

In [16]:
python ../volatility3/vol.py -q -f E:\APT.img windows.svcscan.SvcScan

Volatility 3 Framework 1.2.1

Offset	Order	PID	Start	State	Type	Name	Display	Binary

0x381e90	1	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	Abiosdsk	Abiosdsk	N/A
0x381f20	2	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	abp480n5	abp480n5	N/A
0x381fb0	3	N/A	SERVICE_BOOT_START	SERVICE_RUNNING	SERVICE_KERNEL_DRIVER	ACPI	Microsoft ACPI Driver	\Driver\ACPI
0x382038	4	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	ACPIEC	ACPIEC	N/A
0x3820c8	5	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	adpu160m	adpu160m	N/A
0x382158	6	N/A	SERVICE_SYSTEM_START	SERVICE_RUNNING	SERVICE_KERNEL_DRIVER	AFD	AFD	\Driver\AFD
0x3821e0	7	N/A	SERVICE_BOOT_START	SERVICE_RUNNING	SERVICE_KERNEL_DRIVER	agp440	Intel AGP Bus Filter	\Driver\agp440
0x382270	8	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	Aha154x	Aha154x	N/A
0x382300	9	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	aic78u2	aic78u2	N/A
0x382390	10	N/A	SERVICE_DISABLED	SERVICE_STOPPED	S

0x384710	73	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	hpn	hpn	N/A
0x384798	74	N/A	SERVICE_DEMAND_START	SERVICE_RUNNING	SERVICE_KERNEL_DRIVER	HTTP	HTTP	\Driver\HTTP
0x384820	75	N/A	SERVICE_DEMAND_START	SERVICE_STOPPED	SERVICE_WIN32_SHARE_PROCESS	HTTPFilter	HTTP SSL	N/A
0x3848b8	76	N/A	SERVICE_SYSTEM_START	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	i2omgmt	i2omgmt	N/A
0x384948	77	N/A	SERVICE_DISABLED	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	i2omp	i2omp	N/A
0x3849d0	78	N/A	SERVICE_SYSTEM_START	SERVICE_RUNNING	SERVICE_KERNEL_DRIVER	i8042prt	i8042 Keyboard and PS/2 Mouse Port Driver	\Driver\i8042prt
0x384a60	79	N/A	SERVICE_DEMAND_START	SERVICE_STOPPED	SERVICE_WIN32_SHARE_PROCESS	idsvc	Windows CardSpace	N/A
0x384ae8	80	N/A	SERVICE_SYSTEM_START	SERVICE_STOPPED	SERVICE_KERNEL_DRIVER	Imapi	CD-Burning Filter Driver	N/A
0x384b70	81	N/A	SERVICE_DEMAND_START	SERVICE_STOPPED	SERVICE_WIN32_OWN_PROCESS	ImapiService	IMAPI CD-Burning COM Service	N/A
0x384c08	82	N/A	SERVICE_DISABLED	SERVICE_STOP

In [17]:
volatility_2.6_win64_standalone.exe -f E:\APT.img --profile=WinXPSP2x86 svcscan

Volatility Foundation Volatility Framework 2.6
Offset: 0x381e90
Order: 1
Start: SERVICE_DISABLED
Process ID: -
Service Name: Abiosdsk
Display Name: Abiosdsk
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x381f20
Order: 2
Start: SERVICE_DISABLED
Process ID: -
Service Name: abp480n5
Display Name: abp480n5
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x381fb0
Order: 3
Start: SERVICE_BOOT_START
Process ID: -
Service Name: ACPI
Display Name: Microsoft ACPI Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\ACPI

Offset: 0x382038
Order: 4
Start: SERVICE_DISABLED
Process ID: -
Service Name: ACPIEC
Display Name: ACPIEC
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x3820c8
Order: 5
Start: SERVICE_DISABLED
Process ID: -
Service Name: adpu160m
Display Name: adpu160m
Service Type: SERVICE_KERNEL_DRIVER
Service State: SE

Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\Compbatt

Offset: 0x3834f8
Order: 41
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: COMSysApp
Display Name: COM+ System Application
Service Type: SERVICE_WIN32_OWN_PROCESS
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x383588
Order: 42
Start: SERVICE_DISABLED
Process ID: -
Service Name: Cpqarray
Display Name: Cpqarray
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x383618
Order: 43
Start: SERVICE_AUTO_START
Process ID: 1088
Service Name: CryptSvc
Display Name: Cryptographic Services
Service Type: SERVICE_WIN32_SHARE_PROCESS
Service State: SERVICE_RUNNING
Binary Path: C:\WINDOWS\System32\svchost.exe -k netsvcs

Offset: 0x3836a8
Order: 44
Start: SERVICE_DISABLED
Process ID: -
Service Name: dac960nt
Display Name: dac960nt
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x383738
Order: 45
Start: SE

Process ID: -
Service Name: i2omp
Display Name: i2omp
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x3849d0
Order: 78
Start: SERVICE_SYSTEM_START
Process ID: -
Service Name: i8042prt
Display Name: i8042 Keyboard and PS/2 Mouse Port Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\i8042prt

Offset: 0x384a60
Order: 79
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: idsvc
Display Name: Windows CardSpace
Service Type: SERVICE_WIN32_SHARE_PROCESS
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x384ae8
Order: 80
Start: SERVICE_SYSTEM_START
Process ID: -
Service Name: Imapi
Display Name: CD-Burning Filter Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x384b70
Order: 81
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: ImapiService
Display Name: IMAPI CD-Burning COM Service
Service Type: SERVICE_WIN32_OWN_PROCESS
Service State: SE

Service Type: SERVICE_FILE_SYSTEM_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \FileSystem\Mup

Offset: 0x385eb0
Order: 115
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: napagent
Display Name: Network Access Protection Agent
Service Type: SERVICE_WIN32_SHARE_PROCESS
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x385f40
Order: 116
Start: SERVICE_BOOT_START
Process ID: -
Service Name: NDIS
Display Name: NDIS System Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\NDIS

Offset: 0x385fc8
Order: 117
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: NdisTapi
Display Name: Remote Access NDIS TAPI Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x386058
Order: 118
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: Ndisuio
Display Name: NDIS Usermode I/O Protocol
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\Ndisuio

Offse

Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x3873f0
Order: 153
Start: SERVICE_AUTO_START
Process ID: 716
Service Name: ProtectedStorage
Display Name: Protected Storage
Service Type: SERVICE_INTERACTIVE_PROCESS, SERVICE_WIN32_SHARE_PROCESS
Service State: SERVICE_RUNNING
Binary Path: C:\WINDOWS\system32\lsass.exe

Offset: 0x387490
Order: 154
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: PSched
Display Name: QoS Packet Scheduler
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x387520
Order: 155
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: Ptilink
Display Name: Direct Parallel Link Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x3875b0
Order: 156
Start: SERVICE_DISABLED
Process ID: -
Service Name: ql1080
Display Name: ql1080
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x387640
Order: 157
Start: SERVICE_DISABLED
Proce


Offset: 0x388840
Order: 189
Start: SERVICE_DISABLED
Process ID: -
Service Name: Simbad
Display Name: Simbad
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x3888d0
Order: 190
Start: SERVICE_DISABLED
Process ID: -
Service Name: Sparrow
Display Name: Sparrow
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x388960
Order: 191
Start: SERVICE_AUTO_START
Process ID: 1512
Service Name: Spooler
Display Name: Print Spooler
Service Type: SERVICE_INTERACTIVE_PROCESS, SERVICE_WIN32_OWN_PROCESS
Service State: SERVICE_RUNNING
Binary Path: C:\WINDOWS\system32\spoolsv.exe

Offset: 0x3889f0
Order: 192
Start: SERVICE_BOOT_START
Process ID: -
Service Name: sr
Display Name: System Restore Filter Driver
Service Type: SERVICE_FILE_SYSTEM_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \FileSystem\sr

Offset: 0x388a78
Order: 193
Start: SERVICE_AUTO_START
Process ID: 1088
Service Name: srservice
Display Name: System Res

Service Name: vmci
Display Name: VMware VMCI Bus Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_STOPPED
Binary Path: -

Offset: 0x389d68
Order: 227
Start: SERVICE_SYSTEM_START
Process ID: -
Service Name: vmdebug
Display Name: VMware Replay Debugging Helper
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\vmdebug

Offset: 0x389df8
Order: 228
Start: SERVICE_SYSTEM_START
Process ID: -
Service Name: vmhgfs
Display Name: vmhgfs
Service Type: SERVICE_FILE_SYSTEM_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \FileSystem\vmhgfs

Offset: 0x389e88
Order: 229
Start: SERVICE_AUTO_START
Process ID: -
Service Name: VMMEMCTL
Display Name: Memory Control Driver
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVICE_RUNNING
Binary Path: \Driver\VMMEMCTL

Offset: 0x389f18
Order: 230
Start: SERVICE_DEMAND_START
Process ID: -
Service Name: vmmouse
Display Name: VMware Pointing Device
Service Type: SERVICE_KERNEL_DRIVER
Service State: SERVI

### Shimcache

In [18]:
volatility_2.6_win64_standalone.exe -f E:\APT.img --profile=WinXPSP2x86 shimcache

Volatility Foundation Volatility Framework 2.6
Last Modified                  Last Update                    Path
------------------------------ ------------------------------ ----
2007-11-30 11:18:51 UTC+0000   2009-02-16 23:11:27 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\e76b316b6389286fbb342d033e63f1ba\update\update.exe
2008-04-14 12:00:00 UTC+0000   2009-02-24 20:41:29 UTC+0000   \??\C:\WINDOWS\system32\verclsid.exe
2007-11-30 11:20:44 UTC+0000   2009-02-16 23:10:48 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\f3174104a45ae9b1276d8609df91dcb9\update\update.exe
2008-04-14 12:00:00 UTC+0000   2009-04-13 20:36:12 UTC+0000   \??\C:\WINDOWS\system32\wscntfy.exe
2008-04-14 12:00:00 UTC+0000   2009-02-24 20:41:28 UTC+0000   \??\C:\WINDOWS\System32\cscui.dll
2008-03-20 19:41:20 UTC+0000   2009-02-16 23:10:59 UTC+0000   \??\c:\d9a0b9ff409f8d556fe7\update\update.exe
2008-09-16 19:40:27 UTC+0000   2009-02-16 23:10:57 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Down

2009-01-23 00:24:21 UTC+0000   2009-02-16 23:23:19 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\Install\NDP35SP1-KB958484-x86.exe
2008-12-13 14:56:28 UTC+0000   2009-02-16 23:23:20 UTC+0000   \??\c:\16f9424016f9e0d349c96b7d34dd95\HotFixInstaller.exe
2008-11-15 17:18:04 UTC+0000   2009-02-16 23:23:47 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\e32e42b86ada41fe0c947743c71f222c\update\update.exe
2007-03-06 01:22:59 UTC+0000   2009-02-16 23:24:08 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\21b9c2f7b1db683e3d83bfb825d32092\update\update.exe
2009-02-12 10:56:08 UTC+0000   2009-02-16 23:25:03 UTC+0000   \??\C:\WINDOWS\SoftwareDistribution\Download\Install\windows-kb890830-v2.7.exe
2009-02-12 01:56:18 UTC+0000   2009-02-16 23:25:04 UTC+0000   \??\c:\a76204b5f3f79b9ba2042abb932c2885\mrtstub.exe
2009-02-12 01:56:18 UTC+0000   2009-02-16 23:25:09 UTC+0000   \??\C:\WINDOWS\system32\MRT.exe
2008-09-06 04:29:58 UTC+0000   2009-02-24 20:41:28 UTC+0000   \??\C:\WINDO

## User Activity
### User Assit

In [15]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 userassist

Volatility Foundation Volatility Framework 2.6
----------------------------
Registry: \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT 
Path: Software\Microsoft\Windows\CurrentVersion\Explorer\UserAssist\{0D6D4F41-2994-4BA0-8FEF-620E43CD2812}\Count
Last updated: 2009-02-16 23:02:31 UTC+0000

Subkeys:

Values:

REG_BINARY    UEME_CTLSESSION : Raw Data:
0x00000000  55 84 4c 0e 02 00 00 00                           U.L.....
----------------------------
Registry: \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT 
Path: Software\Microsoft\Windows\CurrentVersion\Explorer\UserAssist\{5E6AB780-7743-11CF-A12B-00AA004AE837}\Count
Last updated: 2009-05-04 22:09:59 UTC+0000

Subkeys:

Values:

REG_BINARY    UEME_CTLSESSION : Raw Data:
0x00000000  28 68 4e 0e 11 00 00 00                           (hN.....

REG_BINARY    UEME_CTLCUACount:ctor : 
ID:             1
Count:          2
Last updated:   1970-01-01 00:00:00 UTC+0000
Raw Data:
0x00000000  01 00 00 00 02 00 00 00 00

### Shellbags

In [16]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 shellbags

Volatility Foundation Volatility Framework 2.6
Scanning for registries....
Gathering shellbag items and building path tree...
***************************************************************************
Registry: \Device\HarddiskVolume1\Documents and Settings\demo\NTUSER.DAT 
Key: Software\Microsoft\Windows\Shell\Bags\1\Desktop
Last updated: 2009-05-05 19:28:35 UTC+0000
Value                     File Name      Modified Date                  Create Date                    Access Date                    File Attr                 Unicode Name
------------------------- -------------- ------------------------------ ------------------------------ ------------------------------ ------------------------- ------------
ItemPos800x600(1)         AGENT-~1.500   2009-04-16 16:10:56 UTC+0000   2009-04-16 14:03:00 UTC+0000   2009-04-16 16:10:56 UTC+0000   DIR                       Agent-1.3.500-SIGNED 
ItemPos800x600(1)         AGENT-~1.900   2009-05-04 22:09:40 UTC+0000   2009-05-04 22:08:24 UTC+0000